In [1]:
from google.colab import files
files.upload()  # upload kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"umejesifavour","key":"9003d9d1683471833eb9d4204c5d5ab9"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c statoil-iceberg-classifier-challenge -p /content/

 55% 159M/288M [00:00<00:00, 1.66GB/s]
100% 288M/288M [00:00<00:00, 1.67GB/s]


In [4]:
!unzip /content/statoil-iceberg-classifier-challenge.zip -d /content/iceberg_data

Archive:  /content/statoil-iceberg-classifier-challenge.zip
  inflating: /content/iceberg_data/sample_submission.csv.7z  
  inflating: /content/iceberg_data/test.json.7z  
  inflating: /content/iceberg_data/train.json.7z  


In [5]:
!apt-get install -y p7zip-full
!7z x /content/iceberg_data/train.json.7z -o/content/iceberg_data/
!7z x /content/iceberg_data/test.json.7z -o/content/iceberg_data/
!7z x /content/iceberg_data/sample_submission.csv.7z -o/content/iceberg_data/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/iceberg_data/                                1 file, 44932785 bytes (43 MiB)

Extracting archive: /content/iceberg_data/train.json.7z
--
Path = /content/iceberg_data/train.json.7z
Type = 7z
Physical Size = 44932785
Headers Size = 154
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      6% - data/processed/train.json                                 12% - data/processed/train.json

In [6]:
!ls -lh /content/iceberg_data/data/processed

total 1.7G
-rw-r--r-- 1 root root 116K Oct 23  2017 sample_submission.csv
-rw-r--r-- 1 root root 1.5G Oct 23  2017 test.json
-rw-r--r-- 1 root root 188M Oct 23  2017 train.json


In [14]:
import pandas as pd

train = pd.read_json("/content/iceberg_data/data/processed/train.json")
test = pd.read_json("/content/iceberg_data/data/processed/test.json")

# Basic info
print("Test shape:", train.shape)
print("Columns:", train.columns)
print(test.head())

# Check missing values
print("\nMissing values per column:")
print(test.isnull().sum())

Test shape: (1604, 5)
Columns: Index(['id', 'band_1', 'band_2', 'inc_angle', 'is_iceberg'], dtype='object')
         id                                             band_1  \
0  5941774d  [-15.863251, -15.201077, -17.887735, -19.17248...   
1  4023181e  [-26.058969497680664, -26.058969497680664, -26...   
2  b20200e4  [-14.14109992980957, -15.064241409301758, -17....   
3  e7f018bb  [-12.167478, -13.706167, -16.54837, -13.572674...   
4  4371c8c3  [-23.37459373474121, -26.02718162536621, -28.1...   

                                              band_2  inc_angle  
0  [-21.629612, -21.142353, -23.908337, -28.34524...  34.966400  
1  [-25.754207611083984, -25.754207611083984, -25...  32.615072  
2  [-14.74563980102539, -14.590410232543945, -14....  37.505433  
3  [-24.32222, -26.375538, -24.096739, -23.8769, ...  34.473900  
4  [-25.72234344482422, -27.011577606201172, -23....  43.918874  

Missing values per column:
id           0
band_1       0
band_2       0
inc_angle    0
dtype: int6

In [15]:
import torch
import numpy as np
import pandas as pd
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import pickle
from tqdm import tqdm

def convert_2channel_to_rgb(img_array):
    """
    Convert a 75×75×2 image to RGB format for ViT.

    Args:
        img_array: NumPy array of shape (75, 75, 2)

    Returns:
        NumPy array of shape (75, 75, 3) in uint8 format
    """
    if img_array.ndim == 2:
        # If 2D, add channel dimension
        img_array = np.expand_dims(img_array, axis=-1)
        img_array = np.concatenate([img_array, img_array], axis=-1)

    # Handle 2-channel images
    if img_array.shape[-1] == 2:
        channel_0 = img_array[:, :, 0]
        channel_1 = img_array[:, :, 1]
        # Create third channel as average
        channel_2 = (channel_0.astype(np.float32) + channel_1.astype(np.float32)) / 2

        # Stack channels
        img_array_rgb = np.stack([channel_0, channel_1, channel_2], axis=-1)
    elif img_array.shape[-1] == 1:
        # Single channel - replicate to 3 channels
        img_array_rgb = np.repeat(img_array, 3, axis=-1)
    else:
        # Already 3+ channels
        img_array_rgb = img_array[:, :, :3]

    # Normalize to [0, 255] range
    if img_array_rgb.max() <= 1.0:
        img_array_rgb = (img_array_rgb * 255).astype(np.uint8)
    else:
        # Normalize to 0-255 if values are larger
        img_min = img_array_rgb.min()
        img_max = img_array_rgb.max()
        if img_max > img_min:
            img_array_rgb = ((img_array_rgb - img_min) / (img_max - img_min) * 255).astype(np.uint8)
        else:
            img_array_rgb = img_array_rgb.astype(np.uint8)

    return img_array_rgb


def prepare_iceberg_image(band1, band2):
    """
    Prepare iceberg image from two bands.

    Args:
        band1: List or array of 5625 values (75*75)
        band2: List or array of 5625 values (75*75)

    Returns:
        NumPy array of shape (75, 75, 2)
    """
    # Convert to numpy arrays and reshape to 75x75
    band1_arr = np.array(band1).reshape(75, 75)
    band2_arr = np.array(band2).reshape(75, 75)

    # Stack as 2 channels
    img_2channel = np.stack([band1_arr, band2_arr], axis=-1)

    return img_2channel


def extract_vit_embeddings_from_dataframe(test_df, output_file='test_embeddings.pkl',
                                          model_name='google/vit-base-patch16-224',
                                          batch_size=32,
                                          band1_col='band_1',
                                          band2_col='band_2',
                                          id_col='id'):
    """
    Extract embeddings from iceberg test DataFrame using pre-trained ViT model.

    Args:
        test_df: Pandas DataFrame with band_1, band_2 columns
        output_file: Output pickle file path
        model_name: HuggingFace model identifier
        batch_size: Batch size for processing
        band1_col: Name of band 1 column
        band2_col: Name of band 2 column
        id_col: Name of ID column (if exists)

    Returns:
        embeddings: NumPy array of embeddings
        ids: List of image IDs
    """

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # DataFrame info
    print(f"\nDataFrame shape: {test_df.shape}")
    print(f"Columns: {test_df.columns.tolist()}")

    # Check if required columns exist
    if band1_col not in test_df.columns or band2_col not in test_df.columns:
        print(f"\nAvailable columns: {test_df.columns.tolist()}")
        raise ValueError(f"Required columns '{band1_col}' and '{band2_col}' not found in DataFrame")

    # Load pre-trained ViT model and processor
    print(f"\nLoading model: {model_name}")
    processor = ViTImageProcessor.from_pretrained(model_name)
    model = ViTModel.from_pretrained(model_name)
    model = model.to(device)
    model.eval()  # Set to inference mode

    num_images = len(test_df)
    print(f"\nProcessing {num_images} images...")

    # Check first image format
    sample_band1 = test_df[band1_col].iloc[0]
    print(f"Sample band_1 type: {type(sample_band1)}")
    print(f"Sample band_1 length: {len(sample_band1) if hasattr(sample_band1, '__len__') else 'N/A'}")

    embeddings = []
    ids = []

    # Process images in batches
    with torch.no_grad():
        for i in tqdm(range(0, num_images, batch_size)):
            batch_end = min(i + batch_size, num_images)

            try:
                # Prepare batch of images
                batch_rgb = []
                batch_ids = []

                for idx in range(i, batch_end):
                    # Get bands
                    band1 = test_df[band1_col].iloc[idx]
                    band2 = test_df[band2_col].iloc[idx]

                    # Prepare 2-channel image
                    img_2channel = prepare_iceberg_image(band1, band2)

                    # Convert to RGB
                    img_rgb = convert_2channel_to_rgb(img_2channel)
                    pil_img = Image.fromarray(img_rgb, mode='RGB')
                    batch_rgb.append(pil_img)

                    # Get ID if exists
                    if id_col in test_df.columns:
                        batch_ids.append(test_df[id_col].iloc[idx])
                    else:
                        batch_ids.append(idx)

                # Preprocess batch
                inputs = processor(images=batch_rgb, return_tensors="pt")
                inputs = {k: v.to(device) for k, v in inputs.items()}

                # Get model outputs
                outputs = model(**inputs)

                # Extract CLS token embeddings (first token)
                cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

                # Store embeddings and IDs
                for emb, img_id in zip(cls_embeddings, batch_ids):
                    embeddings.append(emb)
                    ids.append(img_id)

            except Exception as e:
                print(f"\nError processing batch {i}-{batch_end}: {e}")
                import traceback
                traceback.print_exc()
                continue

    # Convert to numpy array
    embeddings = np.array(embeddings)

    # Save embeddings
    print(f"\nSaving embeddings to {output_file}")
    with open(output_file, 'wb') as f:
        pickle.dump({
            'embeddings': embeddings,
            'ids': ids,
            'model_name': model_name,
            'num_images': num_images
        }, f)

    print(f"\nSuccessfully extracted {len(embeddings)} embeddings")
    print(f"Embedding shape: {embeddings.shape}")

    return embeddings, ids


def load_embeddings(filepath='test_embeddings.pkl'):
    """Load saved embeddings from pickle file."""
    with open(filepath, 'rb') as f:
        data = pickle.load(f)

    print(f"Loaded embeddings from: {filepath}")
    print(f"Model used: {data['model_name']}")
    print(f"Number of embeddings: {len(data['embeddings'])}")
    print(f"Embedding shape: {data['embeddings'].shape}")

    return data['embeddings'], data['ids'], data['model_name']


# Example usage
if __name__ == "__main__":
    # Load your test DataFrame
    test = pd.read_json("/content/iceberg_data/data/processed/test.json")

    print("Test DataFrame loaded:")
    print(f"Shape: {test.shape}")
    print(f"Columns: {test.columns.tolist()}")
    print("\nFirst row:")
    print(test.head(1))

    # Extract embeddings
    OUTPUT_FILE = "test_embeddings.pkl"

    embeddings, ids = extract_vit_embeddings_from_dataframe(
        test_df=test,
        output_file=OUTPUT_FILE,
        model_name='google/vit-base-patch16-224',
        batch_size=32  # Adjust based on GPU memory
    )

    # Verify by loading
    print("\n" + "="*50)
    print("Verifying saved embeddings...")
    loaded_emb, loaded_ids, model = load_embeddings(OUTPUT_FILE)
    print(f"\nFirst 5 IDs: {loaded_ids[:5]}")

Test DataFrame loaded:
Shape: (8424, 4)
Columns: ['id', 'band_1', 'band_2', 'inc_angle']

First row:
         id                                             band_1  \
0  5941774d  [-15.863251, -15.201077, -17.887735, -19.17248...   

                                              band_2  inc_angle  
0  [-21.629612, -21.142353, -23.908337, -28.34524...    34.9664  
Using device: cuda

DataFrame shape: (8424, 4)
Columns: ['id', 'band_1', 'band_2', 'inc_angle']

Loading model: google/vit-base-patch16-224


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Processing 8424 images...
Sample band_1 type: <class 'list'>
Sample band_1 length: 5625


  0%|          | 0/264 [00:00<?, ?it/s]/tmp/ipython-input-3938990263.py:150: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil_img = Image.fromarray(img_rgb, mode='RGB')
100%|██████████| 264/264 [01:49<00:00,  2.40it/s]


Saving embeddings to test_embeddings.pkl

Successfully extracted 8424 embeddings
Embedding shape: (8424, 768)

Verifying saved embeddings...
Loaded embeddings from: test_embeddings.pkl
Model used: google/vit-base-patch16-224
Number of embeddings: 8424
Embedding shape: (8424, 768)

First 5 IDs: ['5941774d', '4023181e', 'b20200e4', 'e7f018bb', '4371c8c3']
